##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 循环神经网络（RNN）文本生成

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://tensorflow.google.cn/tutorials/text/text_generation"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png" />在 tensorflow.google.cn 上查看</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/text/text_generation.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png" />在 Google Colab 运行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/text/text_generation.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png" />在 GitHub 上查看源代码</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/text/text_generation.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png" />下载此 notebook</a>
  </td>
</table>

本教程演示如何使用基于字符的 RNN 生成文本。我们将使用 Andrej Karpathy 在[《循环神经网络不合理的有效性》](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)一文中提供的莎士比亚作品数据集。给定此数据中的一个字符序列 （“Shakespear”），训练一个模型以预测该序列的下一个字符（“e”）。通过重复调用该模型，可以生成更长的文本序列。

请注意：启用 GPU 加速可以更快地执行此笔记本。在 Colab 中依次选择：*运行时 > 更改运行时类型 > 硬件加速器 > GPU*。如果在本地运行，请确保 TensorFlow 的版本为 1.11 或更高。

本教程包含使用 [tf.keras](https://tensorflow.google.cn/programmers_guide/keras) 和 [eager execution](https://tensorflow.google.cn/programmers_guide/eager) 实现的可运行代码。以下是当本教程中的模型训练 30 个周期 （epoch），并以字符串 “Q” 开头时的示例输出：

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

虽然有些句子符合语法规则，但是大多数句子没有意义。这个模型尚未学习到单词的含义，但请考虑以下几点：

* 此模型是基于字符的。训练开始时，模型不知道如何拼写一个英文单词，甚至不知道单词是文本的一个单位。

* 输出文本的结构类似于剧本 -- 文本块通常以讲话者的名字开始；而且与数据集类似，讲话者的名字采用全大写字母。

* 如下文所示，此模型由小批次 （batch） 文本训练而成（每批 100 个字符）。即便如此，此模型仍然能生成更长的文本序列，并且结构连贯。

## 设置

### 导入 TensorFlow 和其他库

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

### 下载莎士比亚数据集

修改下面一行代码，在你自己的数据上运行此代码。

In [3]:
path_to_file = "../Three Kingdoms.txt"

### 读取数据

首先，看一看文本：

In [4]:
# 读取并为 py2 compat 解码
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(text)))

Length of text: 621494 characters


In [5]:
# 看一看文本中的前 250 个字符
print(text[:250])

《三國演義》作者：羅貫中

簡介
　　三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括萬事，七實三虛。三國指的是魏，蜀，吳。小說通篇精巧敘述謀略，被譽為中國謀略全書。
　　羅貫中（1330年一1400年之間），名本，號湖海散人，明代通俗小說家。他的籍貫一說是太原（今山西），一說是錢塘（今浙江杭州），不可確考。據傳說，羅貫中曾充任過元末農民起義軍張士誠的幕客．除《三國誌通俗演義》外，


In [6]:
# 文本中的非重复字符
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

4024 unique characters


## 处理文本

### 向量化文本

在训练之前，我们需要将字符串映射到数字表示值。创建两个查找表格：一个将字符映射到数字，另一个将数字映射到字符。

In [7]:
# 创建从非重复字符到索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

现在，每个字符都有一个整数表示值。请注意，我们将字符映射至索引 0 至 `len(unique)`.

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '.' :   3,
  '0' :   4,
  '1' :   5,
  '2' :   6,
  '3' :   7,
  '4' :   8,
  '5' :   9,
  '6' :  10,
  '7' :  11,
  '8' :  12,
  '9' :  13,
  '?' :  14,
  '[' :  15,
  ']' :  16,
  '—' :  17,
  '…' :  18,
  '□' :  19,
  ...
}


In [9]:
# 显示文本首 13 个字符的整数映射
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'《三國演義》作者：羅貫中\r' ---- characters mapped to int ---- > [  23   33  595 1986 2703   24  139 2727 4021 2694 3253   47    1]


### 预测任务

给定一个字符或者一个字符序列，下一个最可能出现的字符是什么？这就是我们训练模型要执行的任务。输入进模型的是一个字符序列，我们训练这个模型来预测输出 -- 每个时间步（time step）预测下一个字符是什么。

由于 RNN 是根据前面看到的元素维持内部状态，那么，给定此时计算出的所有字符，下一个字符是什么？

### 创建训练样本和目标

接下来，将文本划分为样本序列。每个输入序列包含文本中的 `seq_length` 个字符。

对于每个输入序列，其对应的目标包含相同长度的文本，但是向右顺移一个字符。

将文本拆分为长度为 `seq_length+1` 的文本块。例如，假设 `seq_length` 为 4 而且文本为 “Hello”， 那么输入序列将为 “Hell”，目标序列将为 “ello”。

为此，首先使用 `tf.data.Dataset.from_tensor_slices` 函数把文本向量转换为字符索引流。

In [10]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

《
三
國
演
義


`batch` 方法使我们能轻松把单个字符转换为所需长度的序列。

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'《三國演義》作者：羅貫中\r\n\r\n簡介\r\n\u3000\u3000三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括萬'
'事，七實三虛。三國指的是魏，蜀，吳。小說通篇精巧敘述謀略，被譽為中國謀略全書。\r\n\u3000\u3000羅貫中（1330年一1400年之間），名本，號湖海散人，明代通俗小說家。他的籍貫一說是太原（今山西），一說是錢塘（今'
'浙江杭州），不可確考。據傳說，羅貫中曾充任過元末農民起義軍張士誠的幕客．除《三國誌通俗演義》外，他還創作有《隋唐志傳》等通俗小說和《趙太祖龍虎風雲會》等戲劇。另外，有相當一部分人認為《水滸傳》後三十回也'
'是其所作。\r\n\r\n目錄\r\n\r\n第001回\u3000宴桃園豪傑三結義\u3000斬黃巾英雄首立功 第002回\u3000張翼德怒鞭督郵\u3000何國舅謀誅宦豎 \r\n第003回\u3000議溫明董卓叱丁原\u3000饋金珠李肅說呂布 第004回\u3000廢漢帝陳留踐位\u3000'
'謀董賊孟德獻刀 \r\n第005回\u3000發矯詔諸鎮應曹公\u3000破關兵三英戰呂布 第006回\u3000焚金闕董卓行兇\u3000匿玉璽孫堅背約 \r\n第007回\u3000袁紹磐河戰公孫\u3000孫堅跨江擊劉表 第008回\u3000王司徒巧使連環計\u3000董太師大鬧鳳'


对于每个序列，使用 `map` 方法先复制再顺移，以创建输入文本和目标文本。`map` 方法可以将一个简单的函数应用到每一个批次 （batch）。

In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

打印第一批样本的输入与目标值：

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '《三國演義》作者：羅貫中\r\n\r\n簡介\r\n\u3000\u3000三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括'
Target data: '三國演義》作者：羅貫中\r\n\r\n簡介\r\n\u3000\u3000三國演義是一本長篇歷史小說，可以說是中國古代長篇章回小說的開山之作，亦是四大名著之一。作者是明朝的羅貫中。故事自黃巾起義起，終於西晉統一。是書陳敘百年，賅括萬'


这些向量的每个索引均作为一个时间步来处理。作为时间步 0 的输入，模型接收到 “F” 的索引，并尝试预测 “i” 的索引为下一个字符。在下一个时间步，模型执行相同的操作，但是 `RNN` 不仅考虑当前的输入字符，还会考虑上一步的信息。

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 23 ('《')
  expected output: 33 ('三')
Step    1
  input: 33 ('三')
  expected output: 595 ('國')
Step    2
  input: 595 ('國')
  expected output: 1986 ('演')
Step    3
  input: 1986 ('演')
  expected output: 2703 ('義')
Step    4
  input: 2703 ('義')
  expected output: 24 ('》')


### 创建训练批次

前面我们使用 `tf.data` 将文本拆分为可管理的序列。但是在把这些数据输送至模型之前，我们需要将数据重新排列 （shuffle） 并打包为批次。

In [15]:
# 批大小
BATCH_SIZE = 64

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## 创建模型

使用 `tf.keras.Sequential` 定义模型。在这个简单的例子中，我们使用了三个层来定义模型：

* `tf.keras.layers.Embedding`：输入层。一个可训练的对照表，它会将每个字符的数字映射到一个 `embedding_dim` 维度的向量。 
* `tf.keras.layers.GRU`：一种 RNN 类型，其大小由 `units=rnn_units` 指定（这里你也可以使用一个 LSTM 层）。
* `tf.keras.layers.Dense`：输出层，带有 `vocab_size` 个输出。

In [16]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

对于每个字符，模型会查找嵌入，把嵌入当作输入运行 GRU 一个时间步，并用密集层生成逻辑回归 （logits），预测下一个字符的对数可能性。
![数据在模型中传输的示意图](https://github.com/littlebeanbean7/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)


## 试试这个模型

现在运行这个模型，看看它是否按预期运行。

首先检查输出的形状：

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 4024) # (batch_size, sequence_length, vocab_size)


在上面的例子中，输入的序列长度为 `100`， 但是这个模型可以在任何长度的输入上运行：

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           1030144   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 4024)          4124600   
Total params: 9,093,048
Trainable params: 9,093,048
Non-trainable params: 0
_________________________________________________________________


为了获得模型的实际预测，我们需要从输出分布中抽样，以获得实际的字符索引。这个分布是根据对字符集的逻辑回归定义的。

请注意：从这个分布中 _抽样_ 很重要，因为取分布的 _最大值自变量点集（argmax）_ 很容易使模型卡在循环中。

试试这个批次中的第一个样本：

In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

这使我们得到每个时间步预测的下一个字符的索引。

In [22]:
sampled_indices

array([3471, 1880, 2762, 2181, 2879,  291, 1909, 3563,  720, 3727, 2660,
       1955, 3570, 2812, 2067,  219, 2809, 3261,  240,  629, 2288,  290,
        530, 1633, 3155, 2503, 1154, 2683, 1134, 3916,  964,  148, 1907,
       1697, 1833, 2467,  824, 1345, 2365, 3207, 2214,  499, 3549, 1189,
       1880, 2606, 2786, 3566,  135,  758, 3673, 2967, 3802, 2593, 2258,
       2119, 1040, 1259,  414, 2931, 1928, 3533, 3031,  146, 1960,  845,
       3168, 3586, 1467, 3160, 1141,  130, 3220, 2671, 2541, 1392, 1979,
       2947, 1641, 3458, 3022, 2788, 1725, 2411,   93, 3464, 2039, 3082,
       2163, 2687,  788, 2199, 1039, 1568,  157,  251, 3604, 3224, 1304,
       2806])

解码它们，以查看此未经训练的模型预测的文本：

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '問破黃巾將士索金帛，不從者奏罷職。皇甫嵩、朱俊皆不肯與，趙忠等俱奏罷其官。帝又封趙忠等為車騎將軍，張讓等十三人皆封列侯。朝政愈壞，人民嗟怨。於是長沙賊區星作亂；漁陽張舉、張純反：舉稱天子，純稱大將軍。'

Next Char Predictions: 
 '遷洩肢珪茅凝涼量妃雝繚源釭膽煎傾膠費儲堆癖凜啞架誨竿惑罕悄鬧幟侄涪楊汪穆寅捽瞭譙璧咫醜慓洩絮脯釘何媒降藺頸紳異牢彘扈卸蒼淵酉術佻溢寵諛銜敘調悖低讒纏簞搬滸蕤柔達蠢腐樊祖仁遣灘西獲置孰瑛彎暹侵克鍋谷拗膚'


## 训练模型

此时，这个问题可以被视为一个标准的分类问题：给定先前的 RNN 状态和这一时间步的输入，预测下一个字符的类别。

### 添加优化器和损失函数

标准的 `tf.keras.losses.sparse_categorical_crossentropy` 损失函数在这里适用，因为它被应用于预测的最后一个维度。

因为我们的模型返回逻辑回归，所以我们需要设定命令行参数 `from_logits`。

In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 4024)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       8.300052


使用 `tf.keras.Model.compile` 方法配置训练步骤。我们将使用 `tf.keras.optimizers.Adam` 并采用默认参数，以及损失函数。

In [25]:
model.compile(optimizer='adam', loss=loss)

### 配置检查点

使用 `tf.keras.callbacks.ModelCheckpoint` 来确保训练过程中保存检查点。

In [26]:
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints_2'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### 执行训练

为保持训练时间合理，使用 10 个周期来训练模型。在 Colab 中，将运行时设置为 GPU 以加速训练。

In [27]:
EPOCHS=10

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
96/96 [==============================] - 534s 6s/step - loss: 6.1924
Epoch 2/10
96/96 [==============================] - 508s 5s/step - loss: 5.0730
Epoch 3/10
96/96 [==============================] - 521s 5s/step - loss: 4.6017
Epoch 4/10
96/96 [==============================] - 456s 5s/step - loss: 4.2920
Epoch 5/10
96/96 [==============================] - 272s 3s/step - loss: 4.0607
Epoch 6/10
96/96 [==============================] - 409s 4s/step - loss: 3.8660
Epoch 7/10
96/96 [==============================] - 506s 5s/step - loss: 3.6860
Epoch 8/10
96/96 [==============================] - 506s 5s/step - loss: 3.5192
Epoch 9/10
96/96 [==============================] - 507s 5s/step - loss: 3.3582
Epoch 10/10
96/96 [==============================] - 503s 5s/step - loss: 3.1994


## 生成文本

### 恢复最新的检查点

为保持此次预测步骤简单，将批大小设定为 1。

由于 RNN 状态从时间步传递到时间步的方式，模型建立好之后只接受固定的批大小。

若要使用不同的 `batch_size` 来运行模型，我们需要重建模型并从检查点中恢复权重。

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints_2/ckpt_10'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            1030144   
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 4024)           4124600   
Total params: 9,093,048
Trainable params: 9,093,048
Non-trainable params: 0
_________________________________________________________________


### 预测循环

下面的代码块生成文本：

* 首先设置起始字符串，初始化 RNN 状态并设置要生成的字符个数。

* 用起始字符串和 RNN 状态，获取下一个字符的预测分布。

* 然后，用分类分布计算预测字符的索引。把这个预测字符当作模型的下一个输入。

* 模型返回的 RNN 状态被输送回模型。现在，模型有更多上下文可以学习，而非只有一个字符。在预测出下一个字符后，更改过的 RNN 状态被再次输送回模型。模型就是这样，通过不断从前面预测的字符获得更多上下文，进行学习。

![为生成文本，模型的输出被输送回模型作为输入](https://github.com/littlebeanbean7/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

查看生成的文本，你会发现这个模型知道什么时候使用大写字母，什么时候分段，而且模仿出了莎士比亚式的词汇。由于训练的周期小，模型尚未学会生成连贯的句子。


In [32]:
def generate_text(model, start_string):
  # 评估步骤（用学习过的模型生成文本）

  # 要生成的字符个数
  num_generate = 1000

  # 将起始字符串转换为数字（向量化）
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 空字符串用于存储结果
  text_generated = []

  # 低温度会生成更可预测的文本
  # 较高温度会生成更令人惊讶的文本
  # 可以通过试验以找到最好的设定
  temperature = 1.0

  # 这里批大小为 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # 删除批次的维度
      predictions = tf.squeeze(predictions, 0)

      # 用分类分布预测模型返回的字符
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [34]:
print(generate_text(model, start_string=u"三國 "))

三國 澤欲行此二
　　朱將，自畫良、青阜二人也。來天明召入川中，其掠可宜厚，不久已失矣。「操曰：」討賊之勢，雖毀書，豈容朕孤相惶乎？今使君背義，安敢猶信！「霸又三說本部將士真先去，以為錢糧。趙雲果然鎮南應，深不入境，奔周倉恐眾，百萬發兵而入。懿曰：「某素知兵甚是此；不為將軍相助公瑾，先忘使君，遂立盟於宮醉之間，議論其故，乃聚眾商議，不分兵敗，引數千輛甲水奔浮橋；諸軍在帳中曰：「此四將不保也！」喝令左右推出斬之。眾諸侯悚然。左問楊奉曰：「孔明破山之時，如不孟達；今已失了，軍士不得勝之，意恐未、，非汝二日之盟，只待交鋒，故卻先滅元宵，盡責趙雲、幽州、馬遵英雄將軍法，穩取瀘水之旌。能使人應雖為虎侯，亦是吳侯將趙子龍單、涼等眾謀士人皆厚信之情，恐玄德不定。今曹丕已平拒荊州，可治撫若袁紹，若擊之，必須兩州，猶漢朝廷，若遣一人，使其縛之；其天甚受。長之寄賢，以賓朝廷納降；至輔鄭首，遷都國舅，仰承無名；但等深：生真義於楚矣！「嗣曰：」眾幼間言，便伏皇體，包屨子），拒承正平天病。畢董炎奸虐，糜輅也極善。趙雲出馬即與焦共交戰。斗陣數十餘里，曹操屯住城門，中央梁水打下，一騎刺斜裡到。兩陣對圓，忠亦喝楊松曰：「此非大夫人曾告弟之甥。」荀攸曰：「汝等豈敢抗丞相敵乎？吾見父宗詭之，今若為前攻取長親，早下遠涉而入帳，飲了酒數十杯游著畫清。次日，諸將請樂綝極頌；吳將奐與祿共載：幼和二人，專為貪腹將吏。帝懼之所，不能密意，恐死無怨耳。怎生辨殺之體？必須誤來決計。「
　　
　　施禮畢，阜奏曰：「魏將魏兵甚當城地，不可輕敵。且可當全身只謹從山前去。「王平曰：」敗軍必須擒德。若守潼屍，破劉萇之過矣；隔山破之，必有準備。功勞我築了糧道殺貴，如何？』乃大丈夫，以勵禁愆；我行不服，法正說後也。若不可言，則共攻蜀兵也。」懿問其計。正是：強弱後革未息，供死一旦田。米知勝烈常，文無不學健郡，妙好大書，實與同謀安同諸公州郡。『等不可以為先鋒。」
　　
　　孔明諫曰：「周瑜績喪甚愛人，幸無他限，先遣人敢來求戰未遲。」次曰：「黃公覆虎艱樹，黃三江夏，如雨而降，此必慢之。」
　　
　　二次將眾官捧畢，忽然報鄧艾，後人報知魏王，乃召岑威、金紀商議殺出曹爽以為己連。
　　
　　後槽料知事名，急召回去。懿曰：「吾算度已無一物，今早有細射倒之狀。瓚入帳中見罪，驚問是誰。」漢中王曰：「目恐袁坐初軟多少客，未知，超生不能賢


若想改进结果，最简单的方式是延长训练时间 （试试 `EPOCHS=30`）。

你还可以试验使用不同的起始字符串，或者尝试增加另一个 RNN 层以提高模型的准确率，亦或调整温度参数以生成更多或者更少的随机预测。

## 高级：自定义训练

上面的训练步骤简单，但是能控制的地方不多。

至此，你已经知道如何手动运行模型。现在，让我们打开训练循环，并自己实现它。这是一些任务的起点，例如实现 _课程学习_ 以帮助稳定模型的开环输出。

你将使用 `tf.GradientTape` 跟踪梯度。关于此方法的更多信息请参阅 [eager execution 指南](https://tensorflow.google.cn/guide/eager)。

步骤如下：

* 首先，初始化 RNN 状态，使用 `tf.keras.Model.reset_states` 方法。

* 然后，迭代数据集（逐批次）并计算每次迭代对应的 *预测*。

* 打开一个 `tf.GradientTape` 并计算该上下文时的预测和损失。

* 使用 `tf.GradientTape.grads` 方法，计算当前模型变量情况下的损失梯度。

* 最后，使用优化器的 `tf.train.Optimizer.apply_gradients` 方法向下迈出一步。

In [35]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [36]:
optimizer = tf.keras.optimizers.Adam()

In [37]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [39]:
# 训练步骤
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # 在每个训练周期开始时，初始化隐藏状态
  # 隐藏状态最初为 None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # 每 5 个训练周期，保存（检查点）1 次模型
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 3.0915253162384033
Epoch 1 Loss 3.1516
Time taken for 1 epoch 265.5058150291443 sec

Epoch 2 Batch 0 Loss 2.847195625305176
Epoch 2 Loss 3.0645
Time taken for 1 epoch 267.25120878219604 sec

Epoch 3 Batch 0 Loss 2.643286943435669
Epoch 3 Loss 2.7711
Time taken for 1 epoch 271.30711460113525 sec

Epoch 4 Batch 0 Loss 2.538961887359619
Epoch 4 Loss 2.7476
Time taken for 1 epoch 269.3122022151947 sec

Epoch 5 Batch 0 Loss 2.295781373977661
Epoch 5 Loss 2.5371
Time taken for 1 epoch 265.39928245544434 sec

Epoch 6 Batch 0 Loss 2.209660291671753
Epoch 6 Loss 2.4216
Time taken for 1 epoch 265.9973375797272 sec

Epoch 7 Batch 0 Loss 2.0382533073425293
Epoch 7 Loss 2.3117
Time taken for 1 epoch 266.3118257522583 sec

Epoch 8 Batch 0 Loss 1.892895221710205
Epoch 8 Loss 2.0831
Time taken for 1 epoch 265.13202118873596 sec

Epoch 9 Batch 0 Loss 1.7366843223571777
Epoch 9 Loss 1.9990
Time taken for 1 epoch 263.672646522522 sec

Epoch 10 Batch 0 Loss 1.7016878128051758
Epoch 10

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           1030144   
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 4024)          4124600   
Total params: 9,093,048
Trainable params: 9,093,048
Non-trainable params: 0
_________________________________________________________________


In [51]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints_2/ckpt_9'

In [52]:
model_2 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model_2.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model_2.build(tf.TensorShape([1, None]))

In [53]:
model_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 256)            1030144   
_________________________________________________________________
gru_5 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_5 (Dense)              (1, None, 4024)           4124600   
Total params: 9,093,048
Trainable params: 9,093,048
Non-trainable params: 0
_________________________________________________________________


In [54]:
print(generate_text(model_2, start_string=u"三國 "))

三國 第056回　商議退兵雪，楊彪軍屯於陽興，將淳於丹往來，時苟免連住，不想赤幘，人死者無數。袁紹已知孔明，自賓之欲取西川。往與何主公董後園一句正商議。公瑾舊歲在水中，無此可知孔明昨因星夜墜地，明名童子，夜至府中所管燈著，皆在御林之下，諸葛亮自走，灌上可騎在米倉城內：因此委勝，亦不可及也。宜宜自效良策，實不進心。萬死一辱，使君力盡存薦。望東建一
　　鄉，去三十餘里，正遇選五十番；蠻兵至，乃一群，猶是與魏主否？」眾視之，乃夏侯淵、馬岱、張郃、王封守把各處隘口，把火燒飛，一齊都放起兵來，放火急回，奪路而走。忽然伏路而走，延大路殺來，曹兵大亂，魏軍四散逃走，張郃在後，各自收兵。
　　
　　卻說玄德曰：「果都是也。
　　二人先學所相，指山野而走。雲長急救了一
　　方，船上傍不遠矣。閒人見之人，多有少勞，不可錯憂！」子服曰：「君子遇何高見？」彰戰不三合，被雲一槍刺中面；一將踴奮勇冠，引一小卒，用拖索十餘級，用武二十七子，駕小車曹和，入長安。真英雄也！」於是令休憂。劉表、劉表，杜預理密議，教近臣討之。丁奉表藏先往　
　　
　　
　　
　　卻說當夜二更正江東，聞使雲長提兵於江陵，又令張遼保護。趙雲、文聘、劉瑰、梁虔守把關張，以拒曹操糧草，拍雨渡河，非敵者不曾雙；馬蹄死，吾一軍擁臨陣前妻去，休慌驚走。崔諒見之，提戟橫刀而來，大叫：「丞相在軍中一處射耳。吾頭所以性命，蓋子所恨不安也。「遂設宴款待費禕.飲宴密詔，引兵回官陽平原，大賞三軍。
　　張任是比彝陵人奔回葛陂，大半軍被蜀兵奪了荊州，回報曹睿。魏兵自回
　　人引兵入南進，截路往塵頭息；殺彼首者，因欲堅守，乃彈威教將軍斷雲，也不容矣。」使人去內，百官皆歸求葬。操指其問曰：「汝言老將遠，吾在後時，何故錯耶？」王答曰：「有病車之勇，不亞一載，豈敬若錯？兄欲自縊成都，與天幸作當！
　　持了王，各聞得了官後，獻首數禮，具說呂布轅門都入宮，共了其子，設一宴於會稽扶上殿而歎曰：「害我者無去其厄！」回顧左右將過，來見館驛，細言不答。玄德訴說東南名福，相勸而不感；今魯事為太州，怎能破賊？」當晚而去。昭又將白旗保駕一半，循河為一
　　川，慌辭漢中王。周平大怒，使雲長守護出。維令雲長引軍救住。趙雲引一千餘人，自出魏來，交馬只一合，高定引兩枝兵來，八面殺人；盛德甚是忠心，已知魏國之眾，來救謝天下罪。」漢中王曰：「陛下沮掌社稷，實欲討賊
